In [ ]:
import initialize_notebook # noqa

# OpenAI Python Library

In [ ]:
import os

import openai

client = openai.Client(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=os.environ["GOOGLE_API_KEY"],
)

history = [
    {"role": "system", "content": "You are a helpful assistant tasked with greeting users."},
    {"role": "user", "content": "Hello! My name is Vincent!"},
]

response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=history,
)
print(response)

In [ ]:
import random

choice = random.choice(response.choices)
message = choice.message
print(message)

# Chat Loop

In [ ]:
while True:
    user_input = input("User:")
    history.append({"role": "user", "content": user_input})
    response = client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=history,
    )
    choice = random.choice(response.choices)
    message = choice.message
    print("Assistant:", message.content)
    history.append(message)

# Structured Output

In [ ]:
email = """## 📧 Summit Agenda Preview: "Innovate 2026"

Subject: Your Guide to the **Innovate 2026 Summit** – Agenda & Key Focus Areas

Dear Summit Attendee,

We are delighted to welcome you to the **Innovate 2026 Global Summit**, a dedicated week focusing on the future of sustainable technology and cross-sector collaboration. Below is a preview of the week's structure and thematic focus.

---

### 📅 Thematic Overview: November 10th – November 14th

#### **Monday: The Foundation & Vision**
* **Morning Focus:** Official Welcome and Keynote Address by Dr. Lena Hanson on "**The Next Decade of Disruptive Innovation**." This session will set the global context for the summit.
* **Afternoon Activity:** Structured networking sessions (Sector-Specific Roundtables) designed to foster initial connections and identify shared challenges within your primary industry.

#### **Tuesday & Wednesday: Deep Dives & Collaboration**
* **Tuesday:** Dedicated to **Sustainable Supply Chains and Resource Optimization**. The day will feature a blend of technical workshops on $AI$-driven logistics and panel discussions exploring circular economy models, culminating in the "Green Tech Pitch Challenge" in the evening.
* **Wednesday:** Focus shifts to **Digital Transformation and Ethical $AI$ Implementation**. Attendees will choose from tracks covering data privacy regulation, quantum computing's potential impact, and hands-on demonstrations of new $AR$/$VR$ collaboration platforms.

#### **Thursday: Cross-Sector Solutions & Policy**
* **All Day:** **Interdisciplinary Think Tanks**. Participants will be grouped across different sectors (e.g., Energy, Finance, Healthcare) to develop concrete policy proposals for bridging technological gaps. Key sessions include a debate on global carbon tax frameworks and regulatory harmonization.
* **Evening Gala:** The annual **Visionary Leaders Reception** – a formal event designed for high-level networking and recognition of industry achievements.

#### **Friday: Synthesis & Next Steps**
* **Morning Session:** Presentation of the Thursday Think Tank conclusions to a panel of policy advisors. This is the synthesis phase, converting ideas into actionable plans.
* **Closing Ceremony:** A forward-looking address on "**Translating Summit Insights into Real-World Impact**," followed by an opportunity to commit to one of the week’s proposed collaborative projects. The summit concludes by 1:00 PM.
"""


In [ ]:
instructions = "Extract all the events given in the text provided by the user."

response = client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=[{"role": "system", "content": instructions},
                  {"role": "user", "content": email}],
    )
choice = random.choice(response.choices)
message = choice.message
events = message.content
print(events)

In [ ]:
from pydantic import dataclasses


@dataclasses.dataclass
class Date:
    day: int
    month: int
    year: int

@dataclasses.dataclass
class Time:
    hours: int
    minutes: int
    seconds: int

@dataclasses.dataclass
class Event:
    start_date: Date
    start_time: Time
    end_date: Date
    end_time: Time
    description: str

@dataclasses.dataclass
class Events:
    events: list[Event]

response = client.chat.completions.parse(
        model="gemini-2.5-flash",
        messages=[{"role": "system", "content": instructions},
                  {"role": "user", "content": email}],
        response_format=Events,
    )
choice = random.choice(response.choices)
message = choice.message
events = message.parsed
print(events)

# Jinja Templates

In [ ]:
import dataclasses

import jinja2


@dataclasses.dataclass
class File:
    content: str
    filename: str | None = None



template_string = """Please list all the functions defined in the following modules, with their absolute names (full file path without extension followed by .<function>)

# Files:
{%+ for file in files -%}
{%+ if file.filename -%}
Filename: {{ file.filename }}
{%- else -%}
Filename: __main__
{%- endif +%}
{{ file.content }}
{%+ endfor -%}
"""

template = jinja2.Template(template_string)

In [ ]:
file_1 = File(
    filename="foo/bar.py",
    content="""
def baz():
    return 0

def test(a, b):
    return a + b
""",
)

file_2 = File(
    content="""
from foo import bar

def main():
    return bar.test(bar.baz(), bar.baz())
""")

print(template.render(files=[file_1, file_2]))

In [ ]:
response = client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=[{"role": "user", "content": template.render(files=[file_1, file_2])}],
)
choice = random.choice(response.choices)
message = choice.message
print(message.content)